In [28]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam

from keras.applications.vgg16 import VGG16 # load VGG16 model from keras
from keras.preprocessing.image import load_img # Using images library
from keras.preprocessing.image import img_to_array # # convert image pixels to numpy for specific manipulations
from keras.applications.vgg16 import preprocess_input # to prepare for new input
from keras.applications.vgg16 import decode_predictions # for reporting probabilities

In [29]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
# prepare an iterators for each dataset
train_it = train_datagen.flow_from_directory('data/images/train',
                                     target_size=(224,224),
                                     color_mode='rgb',
                                     batch_size=32,
                                     class_mode='categorical',
                                     shuffle=True)
# val_it = datagen.flow_from_directory('data/validation/', class_mode='categorical')
test_it = train_datagen.flow_from_directory('data/images/test',
                                     target_size=(224,224),
                                     color_mode='rgb',
                                     batch_size=32,
                                     class_mode='categorical',
                                     shuffle=True)
# confirm the iterator works
train_batchX, train_batchy = train_it.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (train_batchX.shape, train_batchX.min(), train_batchX.max()))
test_batchX, test_batchy = test_it.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (test_batchX.shape, test_batchX.min(), test_batchX.max()))

Found 58 images belonging to 5 classes.
Found 16 images belonging to 5 classes.
Batch shape=(32, 224, 224, 3), min=-123.680, max=151.061
Batch shape=(16, 224, 224, 3), min=-123.680, max=151.061


In [30]:
model = VGG16(weights="imagenet")

In [32]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

step_size_train=train_it.n//train_it.batch_size
model.fit_generator(generator=train_it,
                   steps_per_epoch=step_size_train,
                   epochs=1)

Epoch 1/1


ValueError: Error when checking target: expected predictions to have shape (1000,) but got array with shape (5,)